In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# Myopic Pathway Planning

In this example, we demonstrate how to use PyPSA to plan a myopic pathway for emission reductions. That means that we optimize the system for each investment period sequentially subject to set emission reduction targets per period. In consequence, the model does not foresee future costs or benefits of investments. This is in contrast to a [pathway optimisation with perfect foresight](), where the model optimizes the system for all investment periods at once.

Here, we build a model from the [single-node capacity expansion example]() and model a myopic decision process for the investment periods 2025, 2035, and 2045. The model is set up to reduce emissions from 30 MtCO2/a in 2025 to 10 MtCO2/a in 2035 and 3 MtCO2/a in 2045. Note that the goal here is not necessarily to showcase a realistic scenario, but rather to demonstrate how it is possible to model a myopic decision process in PyPSA.

In [ ]:
import pandas as pd

import pypsa
from pypsa.common import annuity

temp = pypsa.examples.model_energy()
temp.remove("Generator", "load shedding")
n = temp.copy()

INVESTMENT_PERIODS = [2025, 2035, 2045]
SOLVER_NAME = "highs"

First, we add a new component for each investment period and carrier to the network. These components are used to model the individual investment decisions for each period. For instance, for solar, there will be a `Generator` component for each investment period, e.g. `solar-2025`, `solar-2035`, and `solar-2045` with the corresponding `build_year` attributes.

We can also keep pre-existing capacities as non-extendable components. Here, we keep 5 GW each of wind and solar capacities, assuming 2015 as build year.

In [ ]:
# Assume 5 GW of wind and solar capacity each, built in 2015
n.generators.loc[:, ["build_year", "p_nom", "p_nom_extendable"]] = 2015, 5_000, False

# all other storage components need to be removed
n.remove("Link", n.links.index)
n.remove("StorageUnit", n.storage_units.index)
n.remove("Store", n.stores.index)

for year in INVESTMENT_PERIODS:
    n.add(
        "Generator",
        temp.generators.index + f"-{year}",
        build_year=year,
        p_max_pu=temp.generators_t.p_max_pu.rename(
            columns=lambda s: s + f"-{year}"
        ).loc[:, ::-1],
        **temp.generators.rename(index=lambda s: s + f"-{year}").drop(
            columns=["build_year", "p_max_pu"]
        ),
    )

    n.add(
        "Link",
        temp.links.index,
        suffix=f"-{year}",
        build_year=year,
        **temp.links.rename(index=lambda s: s + f"-{year}").drop(
            columns=["build_year"]
        ),
    )

    n.add(
        "StorageUnit",
        temp.storage_units.index,
        suffix=f"-{year}",
        build_year=year,
        **temp.storage_units.rename(index=lambda s: s + f"-{year}").drop(
            columns=["build_year"]
        ),
    )

    n.add(
        "Store",
        temp.stores.index,
        suffix=f"-{year}",
        build_year=year,
        **temp.stores.rename(index=lambda s: s + f"-{year}").drop(
            columns=["build_year"]
        ),
    )

In this way, the attributes of components can vary over the investment periods, e.g. the `capital_cost` of solar PV and battery storage can decrease over time, or the capacity factors (`p_max_pu`) of wind generators can increase over time.

In [ ]:
n.generators.loc["solar-2035", "capital_cost"] *= 0.9
n.generators.loc["solar-2045", "capital_cost"] *= 0.85

n.storage_units.loc["battery storage-2035", "capital_cost"] *= 0.9
n.storage_units.loc["battery storage-2045", "capital_cost"] *= 0.85

n.generators_t.p_max_pu.loc[:, "wind-2035"] *= 1.1
n.generators_t.p_max_pu.loc[:, "wind-2045"] *= 1.15

Let us also add some pre-existing conventional generators to the network with emissions from burning fossil fuels.

We add some old coal generators, which will retire by 2035, and some gas generators, which will retire by 2045.
We also let the model invest in new gas generators in 2035, with slightly higher cost but better efficiency.

In [ ]:
n.add(
    "Carrier",
    ["coal", "gas"],
    color=["gray", "lightcoral"],
    co2_emissions=[0.336, 0.198],
)

n.add(
    "Generator",
    "coal",
    carrier="coal",
    bus="electricity",
    p_nom=4_000,
    p_nom_extendable=False,
    build_year=1990,
    lifetime=40,
    capital_cost=annuity(0.07, 40) * 4_000_000,
    efficiency=0.35,
    marginal_cost=30,
)

n.add(
    "Generator",
    "gas",
    carrier="gas",
    bus="electricity",
    p_nom=6_000,
    p_nom_extendable=False,
    build_year=2005,
    lifetime=35,
    capital_cost=annuity(0.07, 35) * 800_000,
    efficiency=0.5,
    marginal_cost=60,
)

n.add(
    "Generator",
    "gas-2035",
    carrier="gas",
    bus="electricity",
    p_nom_extendable=True,
    build_year=2035,
    lifetime=35,
    capital_cost=annuity(0.07, 35) * 900_000,
    efficiency=0.55,
    marginal_cost=55,
)

In the next step, we define the investment periods and the emission reduction targets for each period.
Note that each investment period represents a duration of 10 years, hence the annual targets are scaled to represent total emissions over the entire period.

In [ ]:
n.set_investment_periods(INVESTMENT_PERIODS)
n.investment_period_weightings *= 10

REDUCTION_PATH = [300e6, 100e6, 20e6]

for i, year in enumerate(INVESTMENT_PERIODS):
    n.add(
        "GlobalConstraint",
        f"co2-limit-{year}",
        type="primary_energy",
        carrier_attribute="co2_emissions",
        investment_period=year,
        constant=REDUCTION_PATH[i],
        sense="<=",
    )

To optimize only a single investment period at a time, we have to make a selection of the relevant snapshots (i.e. all that fall into the investment period).
Passing this selection to `n.optimize()` will exclude investment variables of any inactive components (where the build year falls into future investment periods), and also only optimise the dispatch variables for the time steps in the investment period.

In [ ]:
def optimize_period(n: pypsa.Network, period: int):
    snapshots = n.snapshots[n.snapshots.get_level_values("period") == period]
    return n.optimize(
        multi_investment_periods=True,
        solver_name=SOLVER_NAME,
        snapshots=snapshots,
        log_to_console=False,
    )


optimize_period(n, 2025)

display(n.generators.p_nom_opt)

display(n.global_constraints)

In the results we can see that any components after 2025 are not touched in the first investment period.
The CO2 constraint is mildly binding at 20 €/tCO2, which is the price of CO2 in the first period.
The energy mix is still dominated by coal and gas, with some solar and wind added to the system.

In [ ]:
n.statistics.energy_balance().div(1e6).round(2)

For the second investment period, we need to freeze the capacities that were built in the first period, so that they are not changed in the second period (see `freeze_period()` function below). Then we can again select the relevant snapshots and optimize the system for the second investment period -- and so on for any further investment periods.

In [ ]:
def freeze_period(n: pypsa.Network, period: int):
    for c in n.components[["Generator", "Link", "StorageUnit", "Store"]]:
        attr = "e_nom" if c.name == "Store" else "p_nom"
        c.static[attr] = c.static[attr + "_opt"]
        c.static.loc[c.static.build_year == period, attr + "_extendable"] = False


freeze_period(n, 2025)
optimize_period(n, 2035)

freeze_period(n, 2035)
optimize_period(n, 2045)

After the loop has finished, we can inspect the results of all investment periods.
For instance, we can see how the endogenous CO2 price increases with each investment period.

In [ ]:
n.global_constraints

The energy balance and CAPEX statistics also tell us the story that the coal power plants that are phased out in 2035 are largely replaced by a large amount of new gas power plants. Only in 2045, the model starts to double down on renewables and storage investments to meet more ambitious emission reduction targets.
By then, the new gas power plants are barely used, while their capital costs are still to be paid off.

In [ ]:
display(n.statistics.energy_balance().div(1e6).round(1))

display(n.statistics.capex().div(1e6).round(1))

Total annual system costs increase as emissions must be reduced. The data shown does not account for a social discount rate.

In [ ]:
(n.statistics.capex().sum() + n.statistics.opex().sum()).div(1e6).round(1)

Some data wrangling is needed to plot the progression of operational generation capacities over time.

In [ ]:
df = (
    pd.concat(
        {
            year: n.generators.query(
                f"build_year <= {year} and build_year + lifetime > {year}"
            )
            .groupby("carrier")
            .p_nom_opt.sum()
            for year in INVESTMENT_PERIODS
        },
        axis=1,
    )
    .fillna(0)
    .T.div(1e3)
)

df.plot.area(
    stacked=True,
    linewidth=0,
    ylabel="GW",
    xlabel="Year",
    title="Operational Capacity",
    color=df.columns.map(n.carriers.color),
)

**Where to go from here?**

A few ideas to explore further in this notebook:

- Try different exogenous emission reduction paths. How do they affect total costs and cumulative emissions?
- Decrease the capital cost of components over time in different ways. How does this affect the optimal build-out?
- Decrease the capital cost of components over time as a function of the cumulative installed capacity to simulate learning effects.
- Increase the capacity factors of wind and solar components over time to varying extents. How does this affect the optimal build-out?
- Compare the myopic pathway with a full pathway optimization. How do they differ in terms of costs, cumulative emissions, stranded assets? A code snippet is provided blow.

In [ ]:
# for c in n.iterate_components({"Generator", "Link", "StorageUnit", "Store"}):
#     attr = "e_nom" if c.name == "Store" else "p_nom"
#     c.df.loc[c.df.build_year >= 2025, attr + "_extendable"] = True

# n.optimize(
#     multi_investment_periods=True,
#     solver_name="gurobi",
#     log_to_console=True,
# )